In [1]:
# Imports para crear script

import pandas as pd
from datetime import datetime
import json

from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

# Crear conexión con SQLAlchemy
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

clients: pd.DataFrame = pd.read_sql("SELECT vc.* FROM vista_customers vc WHERE vc.codigo_cliente != 0;", engine)

In [3]:
MAIN_DB_HOST = os.getenv("MAIN_DB_HOST")
MAIN_DB_PORT = os.getenv("MAIN_DB_PORT")
MAIN_DB_NAME = os.getenv("MAIN_DB_NAME")
MAIN_DB_USER = os.getenv("MAIN_DB_USER")
MAIN_DB_PASSWORD = os.getenv("MAIN_DB_PASSWORD")

main_engine = create_engine(f"postgresql+psycopg2://{MAIN_DB_USER}:{MAIN_DB_PASSWORD}@{MAIN_DB_HOST}:{MAIN_DB_PORT}/{MAIN_DB_NAME}")


main_clients: pd.DataFrame = pd.read_sql("SELECT c.* FROM clients c;", main_engine)

main_user_clients: pd.DataFrame = pd.read_sql("SELECT uc.* FROM user_clients uc;", main_engine)

In [4]:
CLIENTES = clients[['name', 'codigo_cliente']]

CLIENTES.head(25)

,name,codigo_cliente
0,E Y M CONSTRUCTORES S.A.S.,155
1,CARRIZALES,234
2,WE TALK AI S.A.S.,158
3,INDUSTRIAS ALIMENTICIAS GOURMET S.A.S.,174
4,EXPOFARO S.A.S,184
5,NEURON CONSULTING SAS,187
6,OFICOMCO S.A.S.,225
7,CORPORACION MARYMOUNT DE MEDELLIN,169
8,"PROMOTORA NACIONAL DE CINES S.A.S. ""PROCINAL S...",162
9,FONDO DE EMPLEADOS PARA EL FUTURO FEMFUTURO,233


In [5]:
main_clients.head()

,id,client_code,name
0,1,232,EMCO PARTS S.A.S.
1,2,161,TECNAS S.A. BIC.
2,3,176,COORDINADORA MERCANTIL S.A.
3,5,172,NUESTRA COCINA ARTESANAL S.A.S.
4,6,220,ENERTEM S.A.S.


In [6]:
main_user_clients.head()

,client_id,user_id
0,2,2
1,3,5
2,5,3
3,2,3
4,5,6


In [7]:
def get_new_client_id(codigo_cliente: int) -> int:
    """Función para obtener el nuevo ID del cliente en la base de datos principal."""
    try:
        cliente_row = CLIENTES[CLIENTES['codigo_cliente'] == codigo_cliente]
        return cliente_row.index[0] + 1 if not cliente_row.empty else -1
    except IndexError:
        return -1

In [8]:
def find_old_client_code(new_client_id: int) -> int:
    """Función para encontrar el código de cliente antiguo dado el nuevo ID del cliente."""
    try:
        # find row with client id in main_clients
        client_row = main_clients[main_clients['id'] == new_client_id]
        return client_row['client_code'].values[0] if not client_row.empty else -1
    except IndexError:
        return -1

In [10]:
# for over main_user_clients to update to new client ids
ddl = ["DROP TABLE IF EXISTS user_clients CASCADE;"]

ddl.append("""
CREATE TABLE user_clients (
    user_id int8 NOT NULL,
    client_id int8 NOT NULL,
    CONSTRAINT user_clients_pkey PRIMARY KEY (client_id, user_id),
    FOREIGN KEY (user_id) REFERENCES users(id),
    FOREIGN KEY (client_id) REFERENCES clients(id)
);
""")

ddl.append("""
ALTER TABLE roles DROP CONSTRAINT roles_name_check;
           
ALTER TABLE roles
ADD CONSTRAINT roles_name_check
CHECK (
  name IN ('CONTACT_NORMAL', 'CONTACT_ADMIN', 'ADMIN', 'COMERCIAL')
);
           
INSERT INTO roles(name) VALUES ('COMERCIAL');
""")

for index, row in main_user_clients.iterrows():
    old_client_code = int(find_old_client_code(row['client_id']))
    new_client_id = int(get_new_client_id(old_client_code))
    if new_client_id is None or new_client_id == "NULL":
        print(f"Cliente no encontrado para el código antiguo: {old_client_code}. Saltando inserción.")
        continue
    # updates to new client id
    insert_statement = f"INSERT INTO user_clients (user_id, client_id) VALUES ({row['user_id']}, {new_client_id});"
    ddl.append(insert_statement)

with open("output/update_user_clients.sql", "w", encoding="utf-8") as f:
    f.write("\n".join(ddl))